In [1]:
!pip install -q catboost

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar
import datetime
from datetime import datetime, timedelta, date
import warnings

#settings
warnings.filterwarnings("ignore")


In [3]:
appearances_df = pd.read_csv("../raw_data/appearances.csv")
clubs_df = pd.read_csv("../raw_data/clubs.csv")
games_df = pd.read_csv("../raw_data/games.csv")
players_df = pd.read_csv("../raw_data/players.csv")


In [4]:

features=['goals_2022','goals_against_2022', 'goals_for_2022','games_2022','assists_2022','minutes_played_2022',
          'age','height_in_cm','squad_size','term_days_remaining','position_Attack','position_Defender',
          'position_Goalkeeper','position_Midfield','sub_position_Attacking Midfield','sub_position_Central Midfield',
          'sub_position_Centre-Back','sub_position_Centre-Forward','sub_position_Defensive Midfield',
          'sub_position_Left Midfield','sub_position_Left Winger','sub_position_Left-Back','sub_position_Right Midfield',
          'sub_position_Right Winger','sub_position_Right-Back','sub_position_Second Striker','foot_Left','foot_Right','foot_Both']


In [5]:
def players_df_preproc(players_df):
    #add year to game valuations
    now = datetime.now()
    players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth'])
    players_df = players_df[players_df['date_of_birth'].isnull() == False]
    players_df['age'] = (now - players_df['date_of_birth']).apply(lambda x: x.days) / 365.25
    players_df['age'] = players_df['age'].round().astype(int) 

    # Calculate the contract remaining of each player
    players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])
    players_df = players_df[players_df['contract_expiration_date'].isnull() == False]
    players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days) 
    return players_df

In [6]:
def appearances_df_preproc(appearances_df):
    # add year to player appearances
    appearances_df['datetime']=pd.to_datetime(appearances_df['date'], format="%Y-%m-%d")
    appearances_df['year']=appearances_df['datetime'].dt.year
    appearances_df = appearances_df[(appearances_df.year > 2004 ) & (appearances_df.year < 2023 )]
    return appearances_df

In [7]:
def games_and_appearances_df_preproc(games_df,appearances_df):
    games_df['datetime']=pd.to_datetime(games_df['date'], format="%Y-%m-%d")
    games_df['year']=games_df['datetime'].dt.year
    games_df = games_df[(games_df.year > 2004 ) & (games_df.year < 2023 )]
    games_and_appearances_df = appearances_df.merge(games_df, on=['game_id'], how='left')
    return games_and_appearances_df

In [8]:
#create a function to collate player stats
def player_stats(player_id, season, games_and_appearances_df):
    
    df = games_and_appearances_df[games_and_appearances_df['player_id'] == player_id]
    df =  df[df['season'] == season]    
    if (df.shape[0] == 0):
        Out = [(np.nan, season,0,0,0,0,0,0,0,0,0)]
        out_df = pd.DataFrame(data = Out, columns = ['player_id','season','goals','games',
                                                     'assists','minutes_played','goals_for',
                                                     'goals_against','clean_sheet',
                                                     'yellow_cards','red_cards'])
        return out_df    
    else:       
        df["goals_for"] = df.apply(lambda row: row['home_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['away_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df["goals_against"] = df.apply(lambda row: row['away_club_goals'] if row['home_club_id'] == row['player_club_id'] 
            else row['home_club_goals'] if row['away_club_id'] == row['player_club_id'] 
            else np.nan, axis=1)
        df['clean_sheet'] = df.apply(lambda row: 1 if row['goals_against'] == 0
            else 0 if row['goals_against'] > 0
            else np.nan, axis=1)
        df = df.groupby(['player_id',"season"],as_index=False).agg({'goals': 'sum', 'game_id': 'nunique', 
                                                                    'assists': 'sum', 'minutes_played' : 'sum', 'goals_for' : 'sum',
                                                                    'goals_against' : 'sum', 'clean_sheet' : 'sum','yellow_cards':'sum','red_cards':'sum'})
        out_df = df.rename(columns={'game_id': 'games'})
        return out_df


In [9]:
# preprocessing function to return a dataframe
def preprocessing(clubs_df,players_df,games_and_appearances_df):
    
    merged_players_df=players_df.drop(['current_club_id', 'city_of_birth', 'date_of_birth','first_name', 'last_name', 'player_code', 'image_url', 'url'], axis=1)
    merged_players_df = merged_players_df.reindex(columns = merged_players_df.columns.tolist() + ['club_value','squad_size','goals','goals_2022','games_2022','assists_2022','minutes_played_2022','goals_against_2022','goals_for_2022','clean_sheet_2022'])
    
    for player_id in merged_players_df.player_id.unique():
        club_id = players_df.current_club_id[(players_df.player_id==player_id)]
        try:
            merged_players_df.club_value[(players_df.player_id==player_id)]=int(clubs_df.total_market_value[(clubs_df.club_id==int(club_id))])
        except:
            merged_players_df.club_value[(players_df.player_id==player_id)]='NaN'  
        merged_players_df.squad_size[(players_df.player_id==player_id)]=int((clubs_df.squad_size[(clubs_df.club_id==int(club_id))]))

    columns=['player_id','games_2022','minutes_played_2022','goals_2022','assists_2022','goals_against_2022','goals_for_2022','clean_sheet_2022','name','position','sub_position','last_season','foot','height_in_cm','age','country_of_citizenship','country_of_birth','current_club_name','club_value','squad_size','current_club_domestic_competition_id','agent_name','contract_expiration_date','term_days_remaining','market_value_in_eur','highest_market_value_in_eur']
    merged_players_df=merged_players_df[columns] 

    #iterate through players
    for index in merged_players_df.index:
        id = merged_players_df.loc[index][0]
        name = merged_players_df.loc[index][1]
        
        season = 2022
        stats = player_stats(id, season, games_and_appearances_df)
  
        merged_players_df.at[index,'games_{}'.format(season)]= stats['games'][0]
        merged_players_df.at[index,'goals_{}'.format(season)]= stats['goals'][0]
        merged_players_df.at[index,'assists_{}'.format(season)]= stats['assists'][0]
        merged_players_df.at[index,'minutes_played_{}'.format(season)]= stats['minutes_played'][0]
        merged_players_df.at[index,'goals_for_{}'.format(season)]= stats['goals_for'][0]
        merged_players_df.at[index,'goals_against_{}'.format(season)]= stats['goals_against'][0]
        merged_players_df.at[index,'clean_sheet_{}'.format(season)]= stats['clean_sheet'][0]
        merged_players_df.at[index,'yellow_cards_{}'.format(season)]= stats['yellow_cards'][0]
        merged_players_df.at[index,'red_cards_{}'.format(season)]= stats['red_cards'][0]


    #drop nan
    merged_players_df=merged_players_df.dropna(subset=['market_value_in_eur'])
    merged_players_df1 = merged_players_df[(merged_players_df.current_club_domestic_competition_id=='GB1')]
    
    # convert position categories to Columns for test data
    dummies=pd.get_dummies(merged_players_df1[['position']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 
    
    # convert position categories to Columns for test data
    dummies=pd.get_dummies(merged_players_df1[['sub_position']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 
    dummies=pd.get_dummies(merged_players_df1[['foot']], prefix_sep='_') 
    merged_players_df1 = pd.concat([merged_players_df1, dummies], axis=1) 

    #separate numeric columns
    merged_players_df2 = merged_players_df1[features]
    merged_players_df2['market_value_in_eur'] = merged_players_df1['market_value_in_eur']
    
    return merged_players_df2
    

In [10]:
# test above functions


players_df = players_df_preproc(players_df)
appearances_df = appearances_df_preproc(appearances_df)
games_and_appearances_df = games_and_appearances_df_preproc(games_df,appearances_df)

merged_players_df2 = preprocessing(clubs_df,players_df,games_and_appearances_df)

In [11]:
merged_players_df2.head(4)

,goals_2022,goals_against_2022,goals_for_2022,games_2022,assists_2022,minutes_played_2022,age,height_in_cm,squad_size,term_days_remaining,...,sub_position_Left Winger,sub_position_Left-Back,sub_position_Right Midfield,sub_position_Right Winger,sub_position_Right-Back,sub_position_Second Striker,foot_Left,foot_Right,foot_Both,market_value_in_eur
10,0.0,0.0,0.0,0.0,0.0,0.0,39,188,37.0,283,...,0,0,0,0,0,0,0,1,0,150000.0
11,0.0,0.0,0.0,0.0,0.0,0.0,34,184,37.0,99,...,0,0,0,0,0,0,0,1,0,300000.0
19,0.0,0.0,0.0,0.0,0.0,0.0,36,180,26.0,99,...,0,0,0,0,0,0,0,1,0,500000.0
28,0.0,0.0,0.0,0.0,0.0,0.0,34,180,26.0,99,...,0,0,0,0,1,0,0,1,0,300000.0
